In [18]:
import pandas as pd
import PIL
from glob import glob
import os
import re
import random
import struct
import tensorflow as tf
from tensorflow import keras
import numpy as np

print(tf.__version__)

2.11.0


<font color='red'>Note:</font> Code loosely inspired by [this](https://towardsdatascience.com/board-game-image-recognition-using-neural-networks-116fc876dafa); see also [Github link](https://github.com/andrewleeunderwood/project_MYM)

<font color='red'>--> Does not work yet, only copied from the link above</font>

In [2]:
# laod data:
train_small_df = pd.read_pickle('data/final_data/train/chess_train_df.p')
train_small_df.head()


,filename,width,height,class,xmin,ymin,xmax,ymax,image
13,IMG_20220815_212205_DRO_jpg.rf.006398adca061fc...,224,224,black-bishop,45.422222,107.333328,64.296295,121.17778,"[[[15, 13, 16], [14, 12, 15], [11, 9, 12], [13..."
14,IMG_20220815_212205_DRO_jpg.rf.006398adca061fc...,224,224,black-bishop,46.874073,70.466667,67.40741,85.08889,"[[[15, 13, 16], [14, 12, 15], [11, 9, 12], [13..."
15,IMG_20220815_212205_DRO_jpg.rf.006398adca061fc...,224,224,black-king,13.066667,123.82222,42.725925,136.422226,"[[[15, 13, 16], [14, 12, 15], [11, 9, 12], [13..."
16,IMG_20220815_212205_DRO_jpg.rf.006398adca061fc...,224,224,black-king,10.162963,158.044449,42.725925,179.511108,"[[[15, 13, 16], [14, 12, 15], [11, 9, 12], [13..."
17,IMG_20220815_212205_DRO_jpg.rf.006398adca061fc...,224,224,black-king,162.607407,175.622223,194.340729,202.533325,"[[[15, 13, 16], [14, 12, 15], [11, 9, 12], [13..."


In [14]:
from keras.applications.vgg16 import VGG16
model = VGG16(weights='imagenet')
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
from keras.preprocessing.image import ImageDataGenerator

image_size = (224, 224)
batch_size = 32
folder = 'data/final_data'

datagen = ImageDataGenerator(
        #rotation_range=5,
        rescale=1./255,
        #horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    folder + '/train/train_flow',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle=True)

test_gen = test_datagen.flow_from_directory(
    folder + '/test/test_flow',
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle=False)

Found 2612 images belonging to 12 classes.
Found 301 images belonging to 12 classes.


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3)) 
 
# Freeze convolutional layers from VGG16
for layer in base_model.layers:
    layer.trainable = False
    
# Establish new fully connected block
x = base_model.output
x = Flatten()(x) 
x = Dense(500, activation='relu')(x) 
x = Dense(500, activation='relu')(x)
predictions = Dense(12, activation='softmax')(x)
# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
with tf.device('/cpu:0'):
    epochs = 10
    history = model.fit(
        train_gen, 
        epochs=epochs,
        #verbose = 1,
        validation_data=test_gen)